## Input format:
### File1: dataset/QUT_fish_data/annotations/species_name.pts 
##### PTS file (1 line) containing bounding box:
<bold>if fish facing left:</bold><br>
```bounding_box: <top_x_left>  <top_y_left>; <top_x_right>  <top_y_right>; <bottom_x_right> <bottom_y_right>; <bottom_x_left>  <bottom_y_left> ```

<bold>if fish facing right:</bold><br>
```bounding_box:  <top_x_right>  <top_y_right>; <bottom_x_right> <bottom_y_right>; <bottom_x_left>  <bottom_y_left>; <top_x_left>  <top_y_left>; ```<br>

### File2: dataset/QUT_fish_data/final_all_index.txt 
##### txt file where each line is:

```<class number>=<species name>=<image type>=<full image name>```
## Output format :
### (1 file) where each line is:
 ```image_file_path x_min,y_min,x_max,y_max,class_id```

In [1]:

# returns the coordinates of the corners of an annotation
def get_coord(annotation_path):
    f = open(annotation_path, "r")
    line = f.read()
    
    ### parse lines
    
    # assume fish is facing left
    top_left, top_right, bottom_right, bottom_left = line.split(";")
    temp_top_left_x, temp_top_left_y = top_left.split(':')[1].strip().split("  ")
    temp_top_right_x, temp_top_right_y = top_right.strip().split("  ")
    temp_bottom_right_x, temp_bottom_right_y = bottom_right.strip().split("  ")
    temp_bottom_left_x, temp_bottom_left_y = bottom_left.strip().split(" ")
    
    # prepare return values
    top_left_x = temp_top_left_x;
    top_left_y = temp_top_left_y;
    top_right_x = temp_top_right_x;
    top_right_y = temp_top_right_y;
    bottom_right_x = temp_bottom_right_x;
    bottom_right_y = temp_bottom_right_y;
    bottom_left_x = temp_bottom_left_x;
    bottom_left_y = temp_bottom_left_y;

    
    # if fish is facing right, swap coordinate values
    if float(temp_top_left_x) > float(temp_bottom_right_x): # will only be true if facing right
        top_right_x = temp_top_left_x
        top_right_y = temp_top_left_y
        bottom_right_x = temp_top_right_x
        bottom_right_y = temp_top_right_y
        bottom_left_x = temp_bottom_right_x
        bottom_left_y = temp_bottom_right_y
        top_left_x = temp_bottom_left_x
        top_left_y = temp_bottom_left_y
    
    f.close()
    return (float(top_left_x), float(top_left_y)), (float(top_right_x), float(top_right_y)),(float(bottom_right_x), float(bottom_right_y)), (float(bottom_left_x), float(bottom_left_y))

In [2]:
coords = get_coord(r"keras-yolo3\dataset\QUT_fish_data\annotations\acanthaluteres_vittiger_5.pts")
coords

((476.164271556, 80.8933597621),
 (522.964271556, 235.693359762),
 (165.996085233, 343.613974232),
 (119.196085233, 188.813974232))

In [3]:
# change corner coordinate format to box format
def get_box(coords):
    top_left, top_right, bottom_right, bottom_left = coords
    min_x = min(top_left[0], bottom_left[0])
    max_x = max(top_right[0], bottom_right[0])
    min_y = min(top_left[1], bottom_left[1])
    max_y = max(top_right[1], bottom_right[1])
    return int(min_x), int(max_x), int(min_y), int(max_y)

In [4]:
get_box(coords)

(119, 522, 80, 343)

In [5]:
def parse_species(line):
    class_no, species_name, img_type, img_name, idx = line.split("=")
    return species_name, img_type, img_name

In [6]:
def generate_yolo_annotations():
    species = open(r"keras-yolo3/dataset/QUT_fish_data/final_all_index.txt ","r")
    row_box_annotations = open(r"keras-yolo3/train.txt","w")
    classes = open(r"keras-yolo3/dataset/fish_classes.txt", "w")
    classes_map = {};
    class_no = -1
    for line in species.readlines():
        species_name, img_type, img_name = parse_species(line)
        if(img_type=='insitu'):
            if species_name not in classes_map:
                classes.write(f"{species_name}\n")
                classes_map[species_name] = True
                class_no+=1
            img_path = f"dataset/QUT_fish_data/images/raw_images/{img_name}.jpg"
            annotation_path = f"keras-yolo3/dataset/QUT_fish_data/annotations/{img_name}.pts"
            coords = get_coord(annotation_path)
            min_x,max_x,min_y,max_y = get_box(coords)
            row_box_annotations.write(f"{img_path} {min_x},{min_y},{max_x},{max_y},{class_no}\n")
            
    species.close()
    row_box_annotations.close()
    classes.close()

In [8]:
generate_yolo_annotations()

## Input format:
### File1: dataset/QUT_fish_data/annotations/species_name.pts 
##### CSV file containing bounding box:
```<image_name>,<xmin>,<ymin>,<xmax>,<ymax>,<label>   ```



### File2: dataset/QUT_fish_data/final_all_index.txt 
##### txt file where each line is:

```<class number>=<species name>=<image type>=<full image name>```
## Output format :
### (1 file) where each line is:
 ```image_file_path x_min,y_min,x_max,y_max,class_id```

In [45]:
def parse_csv_line(line):
    img_name, xmin, ymin, xmax, ymax, label = line.split(',')
    return img_name[1:-1], int(float(xmin)), int(float(ymin)), int(float(xmax)), int(float(ymax)), label.strip()[1:-1]

def parse_species(line):
    class_no, species_name, img_type, img_name, idx = line.split("=")
    return species_name, img_type, img_name

def csv_to_yolo_annot():
    species = open(r"keras-yolo3/dataset/QUT_fish_data/final_all_index.txt ","r")
    csv_path = r'keras-yolo3/dataset/QUT_fish_data/images/test/res/vott-csv-export/htfdx-b-fdgcfbdhx-fdg-export.csv'
    imgs_path =  r'keras-yolo3/dataset/QUT_fish_data/images/test/res'
    row_box_annotations = open(r"keras-yolo3/train2.txt","w")
    csv = open(csv_path,'r')
    csv.readline() # first line contains meta data
    for line in csv.readlines():
        img_name, xmin, ymin, xmax, ymax, label = parse_csv_line(line)
        img_path = imgs_path+'/'+img_name
        row_box_annotations.write(f"{img_path} {xmin},{ymin},{xmax},{ymax},{class_no}\n")
    csv.close()


In [46]:
csv_to_yolo_annot()

NameError: name 'class_no' is not defined